In [14]:
#Подгружаем необходимые библитеки
import pandas as pd
import numpy as np
import random
import datetime as dt
import psycopg2 as db
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

#Импорт параметров подключения к БД
from config import database
#Подключение к БД
conn = db.connect(**database)
cursor = conn.cursor()
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:5432/{database}'.format(**database))

# Функция для загруки справочника АЗС
def prepare_df_azs(conn):

    query = "select * from superset.azs_ref order by azs_id"

    df_azs = pd.read_sql(query, conn)

    return df_azs

df_azs = prepare_df_azs(conn)

# Функция для получения Датафрейма для справочника ТРК
def prepare_df_trk(df_azs):
    df_trk = pd.DataFrame()
    t = 0
    for i in range(len(df_azs)):
        if df_azs.loc[i,'rez_volume']==50000:
            for j in range(8):
                df_trk.loc[t, 'trk_num'] = j+1
                df_trk.loc[t, 'azs_id'] = df_azs.loc[i, 'azs_id']

                t += 1
        elif df_azs.loc[i,'rez_volume']==25000:
            for j in range(4):
                df_trk.loc[t, 'trk_num'] = j+1
                df_trk.loc[t, 'azs_id'] = df_azs.loc[i, 'azs_id']
                t += 1
        else:
            for j in range(2):
                df_trk.loc[t, 'trk_num'] = j+1
                df_trk.loc[t, 'azs_id'] = df_azs.loc[i, 'azs_id']
                t += 1
    df_trk['trk_num'] = df_trk['trk_num'].astype(int)
    df_trk['azs_id'] = df_trk['azs_id'].astype(int)
    return df_trk

df_trk = prepare_df_trk(df_azs)

# Функция для получения Датафрейма для справочника пистолетов
def prepare_df_pump(df_azs, df_trk):
    df_pump = pd.DataFrame()
    t = 0
    r = 1
    for v in range(len(df_azs)):
        n = 1
        # Перебор по всем ТРК на АЗС
        for i in range(len(df_trk[df_trk['azs_id']==v+1])):
            # Перебор по всем пистолетам на АЗС
            for j in range(r, (r + df_azs.loc[v, 'rez_count'])):
                df_pump.loc[t, 'pump_num'] = n
                df_pump.loc[t, 'azs_id'] = df_azs.loc[v, 'azs_id']
                df_pump.loc[t, 'rez_id'] = j
                df_pump.loc[t, 'trk_id'] = i+1
                t+=1
                n+=1
            for j in range(r, (r + df_azs.loc[v, 'rez_count'])):
                df_pump.loc[t, 'pump_num'] = n
                df_pump.loc[t, 'azs_id'] = df_azs.loc[v, 'azs_id']
                df_pump.loc[t, 'rez_id'] = j
                df_pump.loc[t, 'trk_id'] = i+1
                t+=1
                n+=1
        r += df_azs.loc[v, 'rez_count']
    df_pump['pump_num'] = df_pump['pump_num'].astype(int)
    df_pump['azs_id'] = df_pump['azs_id'].astype(int)
    df_pump['rez_id'] = df_pump['rez_id'].astype(int)
    df_pump['trk_id'] = df_pump['trk_id'].astype(int)
    return df_pump

df_pump = prepare_df_pump(df_azs, df_trk)

# Функция для получения датафреймов для справочников смен и графика смен
def prepare_shift_ref(df_azs):
    # Отправка GET-запроса к странице
    url = "http://www.rusinkg.ru/russkij-yazyk/article/42-russkij-yazyk-v-mire/6164-samye-rasprostranennye-russkie-familii-rejting-500"
    response = requests.get(url)

    # Парсинг HTML-страницы с помощью BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    p_tags = soup.find_all('p')
    info_list = [tag.text for tag in p_tags]

    # Разделение строки на список элементов
    elements = info_list[15].split()
    surnames = [element for element in elements if not element.isdigit()]
    # Получение списка с фамилиями
    surnames = [name.rstrip('0123456789') for name in surnames]
    
    # Получение датафрейма для справочника с расписанием смен
    df_schedle_shift = pd.DataFrame()
    for i in range(len(df_azs)):
        df_schedle_shift.loc[i, 'azs_id'] = df_azs.loc[i, 'azs_id']
        df_schedle_shift.loc[i, 'time_start_shift'] = dt.datetime.strftime(dt.datetime.strptime('06:00:00', '%H:%M:%S'), '%H:%M:%S')
        df_schedle_shift.loc[i, 'shift_duration'] = 24
    
    # Получение датафрейма для справочника со сменами
    df_shift = pd.DataFrame()
    t = 0
    for i in range(len(df_azs)):
        for j in range(3):
            df_shift.loc[t, 'azs_id'] = df_azs.loc[i, 'azs_id']
            df_shift.loc[t, 'shift_num'] = j + 1
            df_shift.loc[t, 'main_cashier'] = np.random.choice(surnames, replace=False)
            t+=1
    df_shift['shift_num'] = df_shift['shift_num'].astype(int)
    
    return df_shift, df_schedle_shift

df_shift, df_schedle_shift = prepare_shift_ref(df_azs)

# Удаляем данные из таблицы superset.trk_ref
cursor.execute("TRUNCATE TABLE superset.trk_ref CASCADE")
# Возобновляем последовательность в таблице trk_ref
cursor.execute("ALTER SEQUENCE superset.trk_ref_trk_id_seq RESTART WITH 1")
conn.commit()

# Удаляем данные из таблицы superset.pump_ref
cursor.execute("TRUNCATE TABLE superset.pump_ref CASCADE")
# Возобновляем последовательность в таблице pump_ref
cursor.execute("ALTER SEQUENCE superset.pump_ref_pump_id_seq RESTART WITH 1")
conn.commit()

# Удаляем данные из таблицы superset.schedle_shift_ref
cursor.execute("TRUNCATE TABLE superset.schedle_shift_ref CASCADE")
# Возобновляем последовательность в таблице schedle_shift_ref
cursor.execute("ALTER SEQUENCE superset.schedle_shift_ref_schedle_shift_id_seq RESTART WITH 1")
conn.commit()

# Удаляем данные из таблицы superset.shift_ref
cursor.execute("TRUNCATE TABLE superset.shift_ref CASCADE")
# Возобновляем последовательность в таблице shift_ref
cursor.execute("ALTER SEQUENCE superset.shift_ref_shift_id_seq RESTART WITH 1")
conn.commit()

#Запись полученных таблиц в БД
df_trk.to_sql('trk_ref', engine, if_exists='append', index=False, schema='superset')
df_pump.to_sql('pump_ref', engine, if_exists='append', index=False, schema='superset')
df_shift.to_sql('shift_ref', engine, if_exists='append', index=False, schema='superset')
df_schedle_shift.to_sql('schedle_shift_ref', engine, if_exists='append', index=False, schema='superset')

conn.commit()
conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_8332\3449228119.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_azs = pd.read_sql(query, conn)
